# Extract, Transform, and Load raw data into database
Assumes a format following this template:

*Location Date*

*Employer Name*

*Employer Location*

*Application Title*

*Application Contents*

In [1]:
# Import packages: Operating System Interface, System-specific parameters and functions, 
# Regular expression operations, SQLite3 database, CSV and JSON capability, 
# Unix style pathname pattern expansion, Core tools for working with streams, 
# Statistics from text functions, Date-parsin and manipulation, Natural Language Toolkit
import os, sys, re, sqlite3, csv, json, glob, io
from textstat.textstat import textstat
import dateutil.parser as dparser
from datetime import datetime, timedelta
from nltk.tokenize import sent_tokenize, word_tokenize

# Function for limiting the length of a string
def cap(s, l):
    return s if len(s)<=l else s[0:l-3]+'...'

In [2]:
# Create a database named 'Applications.db', with table named 'applications'.
# Commit table (notably, structure) to database, and close the connection
try:
	db = sqlite3.connect('Applications.db')
	cursor = db.cursor()
	cursor.execute('''
	CREATE TABLE `applications` (
		`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
		`path`	TEXT,
		`language`	TEXT,
		`date`	TEXT,
		`longest_sentence`	TEXT,
		`shortest_sentence`	TEXT,
		`lines`	INTEGER,
		`blanklines`	INTEGER,
		`sentences`	INTEGER,
		`number_words`	INTEGER,
		`avg_wordsize`	INTEGER,
		`common_words`	BLOB,
		`syllable_count`	INTEGER,
		`lexicon_count`	INTEGER,
		`sentence_count`	INTEGER,
		`flesch_reading_ease`	INTEGER,
		`flesch_kincaid_grade`	INTEGER,
		`gunning_fog`	INTEGER,
		`smog_index`	INTEGER,
		`automated_readability_index`	INTEGER,
		`coleman_liau_index`	INTEGER,
		`linsear_write_formula`	INTEGER,
		`dale_chall_readability_score`	INTEGER,
		`readability_consensus`	TEXT
	);
	''')
	db.commit()
	db.close()
except Exception as e:
	db.rollback()
	raise e
finally:
	db.close()

In [3]:
# WARNING: This operation should not be ran on large amounts of files,
# or indeed very large individual files

# Define a location for data, begin folder-traversal
data_path = 'data'
for dirname, dirnames, filenames in os.walk(data_path):
    # Separate English from Norwegian applications (by subfolder: 'Eng' and 'Nor'), and note encoding
    for subdirname in dirnames:
        language = os.path.join(subdirname)
        if language == 'Eng':
            enc = 'ASCII'
        elif language == 'Nor':
            enc = 'ISO-8859-1'
        else:
            enc = 'UTF-8'
        # Traverse .txt files in subfolders
        for filename in glob.glob(os.path.join(data_path+'/'+language, '*.txt')):
            filename = filename.replace('\\', '/')
            # For each file, read its contents as 'data' (with line-breaks) and 'string' (without)
            with io.open(filename, 'r') as f:
                data = f.readlines()
                string = ''.join(data)
                
                # Reset counting mechanisms for each file
                lines = 0
                blanklines = 0
                word_list = []
                cf_dict = {}
                word_dict = {}
                punctuations = [",", ".", "!", "?", ";", ":"]
                sentences = 0
                date = 0
                
                # Try to find a date in the first line, !replaced by later function
                date = dparser.parse(data[0],fuzzy=True)
                # Find sentences and words from 'string'
                sentences_tokenized = sent_tokenize(string)
                word_count_tokenized = lambda sentences_tokenized: len(word_tokenize(sentences_tokenized))
                shortest_sentence = min(sentences_tokenized, key=word_count_tokenized)
                longest_sentence = max(sentences_tokenized, key=word_count_tokenized)
                shortest_sentence = shortest_sentence.replace('\n', ' ').replace('\r', '').replace('  ', ' ')
                longest_sentence = longest_sentence.replace('\n', ' ').replace('\r', '').replace('  ', ' ')
                longest_sentence = cap(longest_sentence, 300)
                
                # Read content, line by line, to determine amount of lines, words, and sentences
                for line in data:
                    lines += 1
                    if line.startswith('\n'):
                        blanklines += 1
                    word_list.extend(line.split())
                    for char in line.lower():
                        cf_dict[char] = cf_dict.get(char, 0) + 1

                for word in word_list:
                    lastchar = word[-1]
                    if lastchar in punctuations:
                        word = word.rstrip(lastchar)
                    word = word.lower()
                    word_dict[word] = word_dict.get(word, 0) + 1

                for key in cf_dict.keys():
                    if key in '.!?':
                        sentences += cf_dict[key]
                
                # Count words, average word size, most common words
                number_words = len(word_list)
                num = float(number_words)
                avg_wordsize = len(''.join([k*v for k, v in word_dict.items()]))/num
                mcw = sorted([(v, k) for k, v in word_dict.items()], reverse=True)
                common_words = json.dumps(mcw, encoding=enc)
                
                # Remove location from first line, read date with language-sensitive format
                raw_date = data[0].strip('Bergen').strip()
                if language == 'Eng':
                    date = datetime.strptime(raw_date, "%d/%m/%Y").date()
                elif language == 'Nor':
                    date = datetime.strptime(raw_date, "%d.%m.%Y").date()
                
                # Connect to database, attempt to save all data from the current file
                try:
                    db = sqlite3.connect('Applications.db')
                    cursor = db.cursor()
                    cursor.execute("SELECT ID FROM  applications WHERE path = ?", (filename.decode(enc),))
                    result = cursor.fetchone()
                    if result is None:
                        print('Saving %s ...'%filename.decode(enc))
                        cursor.execute('''INSERT INTO 
                        applications(path, language, date, longest_sentence, shortest_sentence, lines, blanklines, sentences, number_words, avg_wordsize, common_words, syllable_count, lexicon_count, sentence_count, flesch_reading_ease, flesch_kincaid_grade, gunning_fog, smog_index, automated_readability_index, coleman_liau_index, linsear_write_formula, dale_chall_readability_score, readability_consensus)
                        VALUES(:path, :language, :date, :longest_sentence, :shortest_sentence, :lines, :blanklines, :sentences, :number_words, :avg_wordsize, :common_words, :syllable_count, :lexicon_count, :sentence_count, :flesch_reading_ease, :flesch_kincaid_grade, :gunning_fog, :smog_index, :automated_readability_index, :coleman_liau_index, :linsear_write_formula, :dale_chall_readability_score, :readability_consensus)''',
                        {'path':filename.decode(enc), 'language':language, 'date':date, 'longest_sentence':longest_sentence, 'shortest_sentence':shortest_sentence, 'lines':lines, 'blanklines':blanklines, 'sentences':sentences, 'number_words':number_words, 'avg_wordsize':avg_wordsize, 'common_words':common_words, 'syllable_count':textstat.syllable_count(string), 'lexicon_count':textstat.lexicon_count(string), 'sentence_count':textstat.sentence_count(string), 'flesch_reading_ease':textstat.flesch_reading_ease(string), 'flesch_kincaid_grade':textstat.flesch_kincaid_grade(string), 'gunning_fog':textstat.gunning_fog(string), 'smog_index':textstat.smog_index(string), 'automated_readability_index':textstat.automated_readability_index(string), 'coleman_liau_index':textstat.coleman_liau_index(string), 'linsear_write_formula':textstat.linsear_write_formula(string), 'dale_chall_readability_score':textstat.dale_chall_readability_score(string), 'readability_consensus':textstat.readability_consensus(string)})
                        db.commit()
                        id = cursor.lastrowid
                        print('Saved with ID: %d' % id)
                    else:
                        print('%s already exists with ID %s'%(filename.decode(enc),result[0]))
                except Exception as e:
                    db.rollback()
                    raise e
                finally:
                    db.close()

# Confirm that all files found have been evaluated
print('All operations complete.')

Saving data/Eng/AcademicMinds - Ole Vik.txt ...
Saved with ID: 1
Saving data/Eng/Aon - Ole Vik.txt ...
Saved with ID: 2
Saving data/Eng/Aspire - Ole Vik.txt ...
Saved with ID: 3
Saving data/Eng/CBRE - Ole Vik.txt ...
Saved with ID: 4
Saving data/Eng/Crone Corkill - Ole Vik.txt ...
Saved with ID: 5
Saving data/Eng/Euromonitor - Ole Vik.txt ...
Saved with ID: 6
Saving data/Eng/Experis - Ole Vik.txt ...
Saved with ID: 7
Saving data/Eng/Norwegian Barents Secretariat - Ole Vik.txt ...
Saved with ID: 8
Saving data/Eng/Proctor & Gamble - Ole Vik.txt ...
Saved with ID: 9
Saving data/Eng/Rand, Research Assistant - Ole Vik.txt ...
Saved with ID: 10
Saving data/Eng/SAGE - Ole Vik.txt ...
Saved with ID: 11
Saving data/Eng/Springer - Ole Vik.txt ...
Saved with ID: 12
Saving data/Eng/UiO, PhD Cover Letter - Ole Vik.txt ...
Saved with ID: 13
Saving data/Nor/AdmOrg - Ole Vik.txt ...
Saved with ID: 14
Saving data/Nor/BLD - Ole Vik.txt ...
Saved with ID: 15
Saving data/Nor/Bussarbeiderforeningen - Ole V